# Yvettes Notebook

In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
from bs4 import BeautifulSoup
from env import github_token, github_username
import re
import pandas as pd
import time

In [2]:
    #'search?q=stars%3A%3E0&s=stars&type=Repositories'
    #'search?p=2&q=stars%3A%3E0&s=stars&type=Repositories'
    #'search?p=3&q=stars%3A%3E0&s=stars&type=Repositories'
    
#https://github.com/search?q=stars%3A%3E0&s=stars&type=Repositories
#https://github.com/search?p=100&q=stars%3A%3E0&s=stars&type=Repositories

In [4]:
urls_repo = pd.read_csv('urls.csv', index_col=0)
urls_repo['0']

0                  freeCodeCamp/freeCodeCamp
1                             996icu/996.ICU
2     EbookFoundation/free-programming-books
3        jwasham/coding-interview-university
4                       sindresorhus/awesome
                       ...                  
95                   public-apis/public-apis
96                                 vuejs/vue
97                            facebook/react
98          codecrafters-io/build-your-own-x
99                     tensorflow/tensorflow
Name: 0, Length: 100, dtype: object

In [31]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""


# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = urls_df2[0]

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)

In [37]:
df = scrape_github_data()

In [39]:
len(df)

150

In [41]:
df = pd.DataFrame(df)

In [43]:
df.language.isnull().sum()

47

In [44]:
df.language.value_counts()

JavaScript      22
TypeScript      16
Objective-C     15
Python          13
Astro            9
C++              7
Go               6
Java             4
CSS              2
Ruby             1
Scala            1
Assembly         1
Perl             1
Shell            1
C                1
Dart             1
Vue              1
CoffeeScript     1
Name: language, dtype: int64

In [45]:
22+16+15+13

66

In [30]:
18+10+9+8+7+4+1+1


58

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             100 non-null    object
 1   language         58 non-null     object
 2   readme_contents  100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [ ]:
url = "https://github.com/search?q=stars%3A%3E0&s=stars&type=Repositories"
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
urls = []
for link in soup.find_all('a',class_="v-align-middle"):
    print(link.get('href'))


In [46]:
urls=[]
for i in range(0,100):
    url = f"https://github.com/search?p={i}&q=stars%3A%3E0&s=stars&type=Repositories"
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    time.sleep(2)
    for link in soup.find_all('a',class_="v-align-middle"):
        link = re.sub(r'/', '', link.get('href'), count = 1)
        urls.append(link)
        time.sleep(1)
    

In [47]:
len(urls)

1000

In [49]:
urls =pd.DataFrame(urls)

In [50]:
urls.nunique()

0    853
dtype: int64

In [53]:
urls.to_csv('urls_final.csv')

In [52]:
pd.DataFrame.to_csv('urls_final.csv')

AttributeError: 'str' object has no attribute 'to_frame'

In [8]:
urls_df2 = pd.DataFrame(urls)

In [10]:
urls_df2[0]

0                   freeCodeCamp/freeCodeCamp
1                              996icu/996.ICU
2      EbookFoundation/free-programming-books
3         jwasham/coding-interview-university
4             kamranahmedse/developer-roadmap
                        ...                  
145                    yangfuhai/ASimpleCache
146                       shu223/iOS7-Sampler
147         tiann/understand-plugin-framework
148                 twitter-archive/CocoaSPDY
149                             dmotz/oriDomi
Name: 0, Length: 150, dtype: object